## Assignment 2

⚠️ **Important! Submission Requirements**

1. **Do NOT clear notebook logs or outputs.**  
   Every output cell must remain visible (e.g., print logs, progress logs, summary messages).

2. **The notebook must be *self-contained*.**  
   - You should be able to run it **from top to bottom once** without any manual steps.  
   - No need for any external scripts, config files, or multiple runs.
   - Creating multiple cells/blocks is OK.

3. **Single `.ipynb` implementation.**  
   Do not create separate Python files or folders other than the generated image output directory.

4. **Clean, readable, and well-commented code.**

5. **Submission file name format:**  
   → `studentID_name.zip`  
   (Example: `2025000000_jungbeomlee.zip`)
   - This file should contain `COSE-474-02-Assignment_2.ipynb`.
---


## 1. Build Your Own RNN (Forward Only)
**🎯 Goal**


In this problem, you will implement a simple (vanilla) RNN from scratch in PyTorch and verify that its forward pass matches torch.nn.RNN when they share the same parameters.

You will not train this RNN. The goal is to:

- Understand how the hidden state is updated over time.

- Implement the forward computation manually.

- Run a small sanity check to confirm that your implementation is correct.

We will implement:

1. MyRNNCell: computes a single RNN step $h_t$ from $(x_t, h_{t-1})$.

2. MyRNN: applies MyRNNCell repeatedly over the time dimension.

3. A sanity check: copy weights from nn.RNN to MyRNN and verify that the outputs match.

Please fill in all TODO parts. Do not change function signatures.

In [ ]:
# setup
import torch
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
class MyRNNCell(nn.Module):
    """
    A single vanilla RNN cell:
        h_t = tanh(Wx * x_t + Wh * h_{t-1} + b_h)
    """
    def __init__(self, input_dim: int, hidden_dim: int):
        super().__init__()
        # define your weights, ignore output head (Wyh in our lecture note) for this assignment
        self.Wx = nn.Linear(input_dim, hidden_dim, bias=False) # bias: False
        self.Wh = nn.Linear(hidden_dim, hidden_dim, bias=True) # bias: True

    def forward(self, x_t: torch.Tensor, h_prev: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x_t:   (batch_size, input_dim)
            h_prev:(batch_size, hidden_dim)
        Returns:
            h_t:   (batch_size, hidden_dim)
        """

        # implement your codes to compute h_t from x_t and h_prev
        h_t = torch.tanh(self.Wx(x_t) + self.Wh(h_prev))

        return h_t

In [ ]:
class MyRNN(nn.Module):
    """
    A simple RNN that applies MyRNNCell over a sequence.
    We assume batch_first=True, i.e., input shape is (batch, seq_len, input_dim).
    Initialize h_0 as a zero tensor
    """
    def __init__(self, input_dim: int, hidden_dim: int):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.cell = MyRNNCell(input_dim, hidden_dim)

    def forward(self, x: torch.Tensor, h0: torch.Tensor | None = None):
        """
        Args:
            x:  (batch_size, seq_len, input_dim)
            h0: (batch_size, hidden_dim) initial hidden state, or None
        Returns:
            outputs: (batch_size, seq_len, hidden_dim)
            h_T:     (batch_size, hidden_dim)  # hidden state at last time step
        """
        batch_size, seq_len, _ = x.size()



        outputs = []

        # TODO: iterate over time steps and apply the RNN cell
        # Use h0 if provided, otherwise initialize as zeros
        if h0 is None:
            h_t = torch.zeros(batch_size, self.hidden_dim, device=device)
        else:
            h_t = h0

        for t in range(seq_len):
            x_t = x[:, t, :]
            h_t = self.cell(x_t, h_t)
            outputs.append(h_t)
        outputs = torch.stack(outputs, dim=1)
        h_T = h_t


        return outputs, h_T

## Sanity Check: Compare with torch.nn.RNN

Here we:

- Create a PyTorch nn.RNN and your MyRNN with the same dimensions.

- Copy the parameters from nn.RNN into MyRNN so they represent the same RNN.

- Feed the same input and compare outputs.

If your implementation is correct, the mean absolute difference should be very small (e.g., < 1e-6).

In [ ]:
# Hyperparameters for the sanity check
batch_size = 2
seq_len = 5
input_dim = 4
hidden_dim = 3

# Create PyTorch RNN and your MyRNN
torch_rnn = nn.RNN(input_dim, hidden_dim, batch_first=True, nonlinearity="tanh")
my_rnn    = MyRNN(input_dim, hidden_dim)

# Move to device (optional, but good practice)
torch_rnn = torch_rnn.to(device)
my_rnn    = my_rnn.to(device)

# 1) Copy weights from torch_rnn to my_rnn
with torch.no_grad():
    my_rnn.cell.Wx.weight.copy_(torch_rnn.weight_ih_l0)

    # Copy hidden-to-hidden weights
    my_rnn.cell.Wh.weight.copy_(torch_rnn.weight_hh_l0)

    # Combine the two biases from torch_rnn into one bias in MyRNNCell
    combined_bias = torch_rnn.bias_ih_l0 + torch_rnn.bias_hh_l0
    my_rnn.cell.Wh.bias.copy_(combined_bias)

# 2) Create a random input and initial hidden state
x = torch.randn(batch_size, seq_len, input_dim, device=device)

# torch.nn.RNN expects h0 of shape (num_layers, batch, hidden_dim)
h0_torch = torch.randn(1, batch_size, hidden_dim, device=device)
h0_my    = h0_torch.squeeze(0)  # (batch, hidden_dim)

# 3) Forward pass
out_torch, h_torch = torch_rnn(x, h0_torch)      # out_torch: (B, L, H), h_torch: (1, B, H)
out_my, h_my       = my_rnn(x, h0_my)            # out_my:   (B, L, H), h_my:    (B, H)

# 4) Compare outputs
output_diff = torch.abs(out_torch - out_my).mean().item()
hidden_diff = torch.abs(h_torch.squeeze(0) - h_my).mean().item()

print(f"Mean absolute difference (outputs): {output_diff:.8f}")
print(f"Mean absolute difference (hidden):  {hidden_diff:.8f}")

assert output_diff < 1e-4 and hidden_diff < 1e-4, "RNN implementation does not match torch.nn.RNN!"
print("✅ Sanity check passed! Your MyRNN matches torch.nn.RNN (forward).")

Mean absolute difference (outputs): 0.00000184
Mean absolute difference (hidden):  0.00000239
✅ Sanity check passed! Your MyRNN matches torch.nn.RNN (forward).


## 2. Word-level Language Modeling with LSTM

In this problem, you will implement a simple word-level language model using an LSTM in PyTorch.
You will:

1. Load a text corpus from corpus.txt (Tiny Shakespeare).

2. Tokenize the corpus into words and build a vocabulary.

3. Create a dataset of input–target word sequences for next-word prediction.

4. Implement an LSTM-based language model using nn.Embedding, nn.LSTM, and nn.Linear.

5. Train the model to minimize cross-entropy loss using teacher-forcing.

6. Implement a function to generate text given a prompt.

For an input sequence of length $L$,
- Input: $[w_1, w_2, ..., w_L]$
- Output: $[w_2, w_3, ..., w_{L+1}]$


Requirements:
1. Training loss should be under 0.5.
2. Try multiple inputs to "generate_text" function and see if your model generates feasible texts.

In [ ]:
import io
import os
import math
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Download 'corpus.txt' and upload it to the current directory.

with open("corpus.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("Total characters:", len(text))
print("Sample text:")
print(text[:500])

Total characters: 1115393
Sample text:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [ ]:
# Simple word-level tokenization: split on whitespace.
# (You may modify this if you want, but keep it simple.)

tokens = text.strip().split()
print("Number of tokens:", len(tokens))
print("First 50 tokens:", tokens[:50])

Number of tokens: 202651
First 50 tokens: ['First', 'Citizen:', 'Before', 'we', 'proceed', 'any', 'further,', 'hear', 'me', 'speak.', 'All:', 'Speak,', 'speak.', 'First', 'Citizen:', 'You', 'are', 'all', 'resolved', 'rather', 'to', 'die', 'than', 'to', 'famish?', 'All:', 'Resolved.', 'resolved.', 'First', 'Citizen:', 'First,', 'you', 'know', 'Caius', 'Marcius', 'is', 'chief', 'enemy', 'to', 'the', 'people.', 'All:', 'We', "know't,", 'we', "know't.", 'First', 'Citizen:', 'Let', 'us']


In [ ]:
# Build vocabulary: map each unique word to an integer id.
vocab = sorted(set(tokens))
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

vocab_size = len(vocab)
print("Vocab size:", vocab_size)

Vocab size: 25670


In [ ]:
seq_len = 20
batch_size = 32 # you may change this

class WordLMDataset(Dataset):
    def __init__(self, tokens, seq_len, word_to_idx):
        self.seq_len = seq_len
        self.word_to_idx = word_to_idx

        # Convert entire corpus into indices once.
        self.indices = [self.word_to_idx[w] for w in tokens]

    def __len__(self):
        # number of possible sliding windows

        return len(self.indices) - self.seq_len - 1
        # ============================

    def __getitem__(self, idx):

        # ====== YOUR CODE HERE ======
        x = self.indices[idx : idx + self.seq_len]
        y = self.indices[idx + 1 : idx + self.seq_len + 1]
        x = torch.tensor(x, dtype=torch.long)
        y = torch.tensor(y, dtype=torch.long)
        # ============================

        return x, y

dataset = WordLMDataset(tokens, seq_len, word_to_idx)
print(len(dataset), dataset[0][0].shape, dataset[0][1].shape)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)


202630 torch.Size([20]) torch.Size([20])


In [ ]:
# Define the LSTM Language Model
class WordLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=1):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        """
        Args:
            x:      (batch, seq_len) int64 word indices
            hidden: (h0, c0) or None
        Returns:
            logits: (batch, seq_len, vocab_size)
            hidden: (h_T, c_T)
        """

        # ====== YOUR CODE HERE ======
        embeds = self.embedding(x)

        lstm_out, hidden = self.lstm(embeds, hidden)

        logits = self.fc(lstm_out)

        # ============================

        return logits, hidden

In [ ]:
# Training Loop

embed_dim = 128
hidden_dim = 256
num_layers = 1
num_epochs = 5   # you may increase/decrease
learning_rate = 1e-3

model = WordLSTM(vocab_size, embed_dim, hidden_dim, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for x, y in loader:
        x = x.to(device)   # (B, L)
        y = y.to(device)   # (B, L)

        optimizer.zero_grad()

        # TODO: forward, compute loss, backward, step
        # Remember:
        #   logits: (B, L, V)
        #   CrossEntropyLoss expects: (B*L, V) and targets (B*L,)

        # ====== YOUR CODE HERE ======
        logits, _ = model(x)

        loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))

        loss.backward()
        optimizer.step()

        # ============================

        total_loss += loss.item()

    return total_loss / len(loader)

for epoch in range(1, num_epochs + 1):
    train_loss = train_one_epoch(model, loader, optimizer, criterion, device)
    print(f"Epoch {epoch:02d} | train loss: {train_loss:.4f}")

Epoch 01 | train loss: 4.4023
Epoch 02 | train loss: 1.6476
Epoch 03 | train loss: 0.9239
Epoch 04 | train loss: 0.6778
Epoch 05 | train loss: 0.5756


In [ ]:
def generate_text(model, start_text, max_new_tokens=50, temperature=1.0):
    """
    Args:
        model:      trained WordLSTM
        start_text: string, seed prompt (e.g., "To be or not to be")
        max_new_tokens: how many words to generate
        temperature: softmax temperature (>0). Higher = more random.
    """
    model.eval()

    # Tokenize start_text into words
    words = start_text.strip().split()

    # Map to indices (OOV words are mapped to 0 or some fallback)
    # Here, we simply skip unknown words or map them to 0.
    idxs = []
    for w in words:
        if w in word_to_idx:
            idxs.append(word_to_idx[w])
        else:
            # unknown word → you can choose to skip or map to 0
            # here we map to 0:
            idxs.append(0)

    x = torch.tensor([idxs], dtype=torch.long, device=device)  # (1, L)
    hidden = None

    with torch.no_grad():
        for _ in range(max_new_tokens):
            logits, hidden = model(x, hidden)
            # Focus on the last time step
            last_logits = logits[:, -1, :] / temperature   # (1, V)
            probs = F.softmax(last_logits, dim=-1)         # (1, V)
            next_idx = torch.multinomial(probs, num_samples=1).item()

            words.append(idx_to_word[next_idx])

            # Now feed only the last predicted token as next input
            x = torch.tensor([[next_idx]], dtype=torch.long, device=device)

    return " ".join(words)

In [ ]:
print(generate_text(model, start_text="To be", max_new_tokens=30, temperature=1.0))

To be your comforter when by his lord, lest I the son Were I respected then to do him know his mother was perforce withheld. BUCKINGHAM: Fie, what an indirect and peevish


## 3. Character-level Language Modeling with LSTM

In this problem, you will build **a character-level language model** using PyTorch’s nn.LSTM.
Unlike word-level language modeling, here we treat each character as a token.

You will:

- Build a character vocabulary from the corpus.

- Implement a CharLMDataset that creates (input, target) sequences for next-character prediction.

- Implement a CharLSTM model using nn.Embedding and nn.LSTM.

- Train the model with teacher forcing: at each time step, the model receives the ground-truth previous characters as input.


We will use the same corpus file as in Problem 2.


Requirements:
1. Training loss should be under 1.1.
2. Try multiple inputs to "generate_characters" function and see if your model generates feasible texts.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Load the full corpus as a single string
with open("corpus.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("Total number of characters in corpus:", len(text))
print("Preview:")
print(text[:500])

Total number of characters in corpus: 1115393
Preview:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [ ]:
# Build character vocabulary
chars = sorted(list(set(text)))
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for c, i in char_to_idx.items()}
vocab_size_char = len(chars)

In [ ]:
# Character-level dataset

seq_len_char = 100

class CharLMDataset(Dataset):
    def __init__(self, text: str, seq_len: int):
        super().__init__()
        self.seq_len = seq_len
        # Convert entire text into a list of indices
        self.indices = [char_to_idx[c] for c in text]

    def __len__(self):

        return len(self.indices) - self.seq_len

    def __getitem__(self, idx):

        # ====== YOUR CODE HERE ======
        x = self.indices[idx : idx + self.seq_len]
        y = self.indices[idx + 1 : idx + self.seq_len + 1]
        x = torch.tensor(x, dtype=torch.long)
        y = torch.tensor(y, dtype=torch.long)
        # ============================

        return x, y

dataset_char = CharLMDataset(text, seq_len_char)
batch_size = 64

loader_char = DataLoader(dataset_char, batch_size=batch_size, shuffle=True, drop_last=True)

print("Number of training examples:", len(dataset_char))
x_example, y_example = next(iter(loader_char))
print("Example batch shapes:", x_example.shape, y_example.shape)  # (B, L), (B, L)

Number of training examples: 1115293
Example batch shapes: torch.Size([64, 100]) torch.Size([64, 100])


In [ ]:
# Character-level LSTM model

class CharLSTM(nn.Module):
    def __init__(self, vocab_size: int, embed_dim: int = 128, hidden_dim: int = 256, num_layers: int = 1):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x: torch.Tensor, hidden=None):
        """
        Args:
            x:      (batch_size, seq_len)  integer character indices
            hidden: (h_0, c_0) or None
        Returns:
            logits: (batch_size, seq_len, vocab_size)
            hidden: (h_T, c_T)
        """


        # ====== YOUR CODE HERE ======
        embeds = self.embedding(x)
        out, hidden = self.lstm(embeds, hidden)
        logits = self.fc(out)
        # ============================

        return logits, hidden

In [ ]:
# training

model_char = CharLSTM(vocab_size_char, embed_dim=128, hidden_dim=256, num_layers=1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer_char = torch.optim.Adam(model_char.parameters(), lr=1e-3)

num_epochs = 5  # you may increase or decrease this


def train_char_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for x, y in loader:
        x = x.to(device)   # (B, L)
        y = y.to(device)   # (B, L)

        optimizer.zero_grad()

        # TODO: forward pass, compute loss, backprop, optimizer step


        # ====== YOUR CODE HERE ======
        logits, _ = model(x)

        loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))

        loss.backward()
        optimizer.step()

        # ============================

        total_loss += loss.item()

    return total_loss / len(loader)


for epoch in range(1, num_epochs + 1):
    train_loss = train_char_epoch(model_char, loader_char, optimizer_char, criterion, device)
    print(f"[Char LM] Epoch {epoch}: train loss = {train_loss:.4f}")

[Char LM] Epoch 1: train loss = 1.2638
[Char LM] Epoch 2: train loss = 1.0926
[Char LM] Epoch 3: train loss = 1.0530
[Char LM] Epoch 4: train loss = 1.0362
[Char LM] Epoch 5: train loss = 1.0264


In [ ]:
# Text generation (sampling)

def generate_characters(model, start_text: str, max_len: int = 300, temperature: float = 1.0) -> str:
    """
    Generate text from a trained character-level language model.
    """
    model.eval()
    chars = list(start_text)

    # Convert initial prompt to indices
    idxs = [char_to_idx.get(c, 0) for c in chars]  # unknown chars -> 0
    x = torch.tensor([idxs], dtype=torch.long, device=device)  # (1, L0)

    hidden = None

    with torch.no_grad():
        # First, run the prompt through the model to update hidden state
        if x.size(1) > 0:
            logits, hidden = model(x, hidden)

        # Now, generate new characters step by step
        for _ in range(max_len):
            # Use the last character as input
            last_idx = idxs[-1]
            x_last = torch.tensor([[last_idx]], dtype=torch.long, device=device)  # (1, 1)

            logits, hidden = model(x_last, hidden)  # logits: (1, 1, V)
            last_logits = logits[:, -1, :] / temperature  # (1, V)
            probs = F.softmax(last_logits, dim=-1)        # (1, V)

            # Sample next character index
            next_idx = torch.multinomial(probs, num_samples=1).item()
            idxs.append(next_idx)
            chars.append(idx_to_char[next_idx])

    return "".join(chars)


In [ ]:
print(generate_characters(model_char, start_text="KING ", max_len=300, temperature=0.8))


KING LEWIS XI:
Then give me leave.

ESCALUS:
Go Father, father, thou shalt not the hang
As was-day the seal'd, they well degree,
In condition e'ery look palland when;
How now, that man the blame service shall do it
With hearts in thy mine. How will my mes's old lameness:
Fare ye stars, and desire to impr
